In [21]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import plotly.express as px


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

from yellowbrick.cluster.elbow import kelbow_visualizer

from lifetimes.utils import summary_data_from_transaction_data

In [22]:
# read in the files
portfolio = pd.read_csv('portfolio.csv')
profile = pd.read_csv('in_cart_customer.csv')
transcript = pd.read_csv('transcript.csv')

In [23]:
portfolio.head()

,Unnamed: 0,reward,channels,difficulty,duration,offer_type,id
0,0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [27]:
profile.head(20)

,user_id,gender,age,income,customer score
0,1,Male,55,112000,11
1,3,Male,68,70000,13
2,5,Female,58,51000,5
3,8,Female,62,71000,4
4,12,Female,40,71000,6
5,13,Male,64,100000,13
6,14,Female,78,71000,14
7,15,Male,42,69000,23
8,16,Female,56,88000,7
9,18,Female,46,59000,7


In [25]:
transcript.head()

,Unnamed: 0,person,event,value,time
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [26]:
# Bring back original profile
clean_profile = pd.read_csv('data/clean_profile.csv')
clean_profile.head(20)

,user_id,gender,age,id,income
0,1,Male,55.00000,0610b486422d4921ae7d2bf64640c50b,112000.0
1,3,Male,75.00000,78afa995795e4d85b5d9ceeca43f5fef,100000.0
2,5,Female,68.00000,e2127556f4f64592b11af22de27a7932,70000.0
3,8,Female,65.00000,389bc3fa690240e798340f5a15918d5c,53000.0
4,12,Female,58.00000,2eeac8d8feae4a8cad5a6af0499a211d,51000.0
5,13,Male,61.00000,aa4862eba776480b8bb9c68455b8c2e1,57000.0
6,14,Female,26.00000,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0
7,15,Male,62.00000,31dda685af34476cad5bc968bdb01c53,71000.0
8,16,Female,49.00000,62cf5e10845442329191fc246e7bcea3,52000.0
9,18,Female,57.00000,6445de3b47274c759400cd68131d91b4,42000.0


In [28]:
#merge both profiles so gender is updated
profile = profile[['user_id','gender','age',]].merge(clean_profile[['user_id','id','income']], on='user_id', how='left')

In [29]:
#Make sure gender and 
profile.head()

,user_id,gender,age,id,income
0,1,Male,55,0610b486422d4921ae7d2bf64640c50b,112000.0
1,3,Male,68,78afa995795e4d85b5d9ceeca43f5fef,100000.0
2,5,Female,58,e2127556f4f64592b11af22de27a7932,70000.0
3,8,Female,62,389bc3fa690240e798340f5a15918d5c,53000.0
4,12,Female,40,2eeac8d8feae4a8cad5a6af0499a211d,51000.0
